# Bemobile

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
from datetime import datetime
import keras
from keras import initializers, regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten, Input, concatenate

from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
import keras.backend as K


## Download data from the cloud and load

In [ ]:
# X_train = np.load("../data/train-test/X_train.npy")
y_train = np.load("../data/train-test/y_train.npy")
# X_test = np.load("../data/train-test/X_test.npy")
y_test = np.load("../data/train-test/y_test.npy")
X_train_conv = np.load("../data/train-test/X_train_conv.npy")
X_train_val = np.load("../data/train-test/X_train_val.npy")
X_test_conv = np.load("../data/train-test/X_test_conv.npy")
X_test_val = np.load("../data/train-test/X_test_val.npy")

In [ ]:
def conv_model(data, **params):
    '''
    parameters:
    conv_layers: array with number of convolutions (length is number of layers)
    maxpooling: 1 for maxpooling, 0 for AveragePooling2D 
    poolsize: array with poolsizes (length is number of layers)
    conv_dense: array with number of dense elements after the conv part (length is number of layers)
    val_dense: array with number of dense elements for the metadata part (length is number of layers)
    comb_dense: array with number of dense elements for the combined part (length is number of layers)
    dropout: dropout parameter used equally everywhere (only with dense)
    epochs: epochs to train
    '''
    (X_train_conv, X_train_val, y_train, X_test_conv, X_train_val, y_test) = data
    # define two sets of inputs
    
    X_conv = Input(shape=(60,31,1,))
    X_data = Input(shape=(9,))
    
    f=15
    chanDim = -1
    
    dropout = params['dropout']
    
    #### X
    conv_size = params['conv_layers'][0]
    pool_size = params['pool_size'][0]
    
    x = Conv2D(f, conv_size, padding="same")(X_conv)
    x = BatchNormalization(axis=chanDim)(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=pool_size)(x) if params['maxpooling'] \
    else AveragePooling2D(pool_size=pool_size)(x)

    for i in range(1, len(params['conv_layers'])):
        conv_size = params['conv_layers'][i]
        pool = params['pool_size'][i]

        x = Conv2D(f, conv_size, padding="same")(X_conv)
        x = BatchNormalization(axis=chanDim)(x)
        x = Activation("relu")(x)
        x = MaxPooling2D(pool_size=pool_size)(x) if params['maxpooling'] \
        else AveragePooling2D(pool_size=pool_size)(x)
    
    x = Flatten()(x)
    for i in range(len(params['conv_dense'])):
        dense_size = params['conv_dense'][i]
        
        x = Dense(dense_size)(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Activation("relu")(x)
        x = Dropout(dropout)(x)
        
    x = Model(inputs=X_conv, outputs=x)

    #### Y
    dense_size = params['val_dense'][0]
    y = Dense(dense_size, activation="relu")(X_data)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    
    for i in range(1, len(params['val_dense'])):
        dense_size = params['val_dense'][i]
        
        y = Dense(dense_size)(y)
        y = BatchNormalization()(y)
        y = Activation("relu")(y)
        y = Dropout(dropout)(y)

    y = Model(inputs=X_data, outputs=y)

    #### Z
    # combine the output of the two branches
    combined = concatenate([x.output, y.output])
    # apply a FC layer and then a regression prediction on the
    # combined outputs
    
    dense_size = params['comb_dense'][0]
    z = Dense(dense_size, activation="relu")(combined)
    z = BatchNormalization()(z)
    z = Activation("relu")(z)
    for i in range(len(params['comb_dense'])-1):
        densesize = params['comb_dense'][i+1]
        
        z = Dense(dense_size)(z)
        z = BatchNormalization()(z)
        z = Activation("relu")(z)
        z = Dropout(dropout)(z)
    

    z = Dense(3600, activation="linear")(z)
    
    model = Model(inputs=[x.input, y.input], outputs=z)
    
    model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='mse')    
    
    return model

In [ ]:
params = {'conv_layers': [(10, 10), (10, 10), (5, 5)],
'maxpooling': False,
'pool_size': [(2, 2), (2, 2), (3, 3)],
'conv_dense': [1200, 800],
'val_dense': [10, 8],
'comb_dense': [1500],
'dropout': 0.15,
'epochs': 20} 

In [ ]:
data = [X_train_conv, X_train_val, y_train, X_test_conv, X_train_val, y_test]

In [ ]:
model = conv_model(data, **params)

In [ ]:
history_callback = model.fit(
    x=[X_train_conv, X_train_val],
    y=y_train,
    validation_data = ([X_test_conv, X_test_val], y_test),
    epochs = params['epochs'],
    batch_size = 1024,
    verbose = 1)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(history_callback.history['loss'],label = 'train loss')
plt.plot(history_callback.history['val_loss'], label = 'validation loss')
plt.legend()
plt.show()

In [ ]:
import os

In [ ]:
os.mkdir('../snapshots')

In [ ]:
model.save("../snapshots/bemobile-model.h5")

## Upload this model to the cloud

## Make sure this file is executable in one run